In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt


In [4]:
#pip install statsmodels


In [5]:
data = pd.read_csv('/Users/amulya/Desktop/Capstone/DSCI-601-Amy/Data/FeatureEngineered/AKAM_feature_engineeredv2.csv')

In [6]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [8]:
non_numeric_cols = ['TICKER', 'CUSIP', 'COMNAM']
numeric_cols = data.columns.difference(non_numeric_cols)

data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Forward-fill missing values
data.fillna(method='ffill', inplace=True)
# Backward-fill if any NaNs remain
data.fillna(method='bfill', inplace=True)




/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_83250/3688063530.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_83250/3688063530.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [9]:
data.isnull().sum()

TICKER             0
CUSIP              0
COMNAM             0
RET                0
VOL_CHANGE         0
BA_SPREAD          0
ILLIQUIDITY        0
sprtrn             0
TURNOVER           0
PRC                0
SHROUT             0
MARKET_CAP         0
TRAN_COST          0
ASK                0
BID                0
day_of_week        0
day_of_month       0
month              0
is_month_start     0
is_month_end       0
year               0
week               0
RET_ema_12         0
RET_ema_26         0
RET_skew_10        0
RET_kurtosis_10    0
RET_entropy_10     0
RSI                0
OBV                0
RET_lag_1          0
RET_lag_7          0
RET_lag_14         0
RET_lag_30         0
RET_mean_7d        0
RET_std_7d         0
RET_mean_14d       0
RET_std_14d        0
RET_mean_30d       0
RET_std_30d        0
dtype: int64

In [10]:
endog_vars = ['PRC', 'RET']
exog_vars = data.columns.difference(endog_vars + non_numeric_cols)


In [12]:
model_data = data[endog_vars + list(exog_vars)]


In [13]:
def adf_test(series, title=''):
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['Test Statistic', 'p-value', '# Lags Used', '# Observations']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    print('')

# Apply ADF test to each variable
for column in model_data.columns:
    adf_test(model_data[column], title=column)


Augmented Dickey-Fuller Test: PRC
Test Statistic            -5.601166
p-value                    0.000001
# Lags Used               34.000000
# Observations          6045.000000
Critical Value (1%)       -3.431432
Critical Value (5%)       -2.862018
Critical Value (10%)      -2.567025

Augmented Dickey-Fuller Test: RET
Test Statistic         -1.329064e+01
p-value                 7.374884e-25
# Lags Used             3.200000e+01
# Observations          6.047000e+03
Critical Value (1%)    -3.431432e+00
Critical Value (5%)    -2.862018e+00
Critical Value (10%)   -2.567024e+00

Augmented Dickey-Fuller Test: ASK
Test Statistic            -5.589161
p-value                    0.000001
# Lags Used               34.000000
# Observations          6045.000000
Critical Value (1%)       -3.431432
Critical Value (5%)       -2.862018
Critical Value (10%)      -2.567025

Augmented Dickey-Fuller Test: BA_SPREAD
Test Statistic            -2.453593
p-value                    0.127178
# Lags Used         

In [14]:
model_data_diff = model_data.diff().dropna()


In [15]:
model = VAR(model_data_diff)
lag_order_results = model.select_order(maxlags=4)
print(lag_order_results.summary())


/Users/amulya/.pyenv/versions/3.10.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: 10-th leading minor of the array is not positive definite

In [16]:
model = VAR(model_data_diff)

# Since we have only 4 data points after differencing, use lag_order=1
results = model.fit(1)
print(results.summary())

/Users/amulya/.pyenv/versions/3.10.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: 10-th leading minor of the array is not positive definite

# Debugging

In [18]:
# Use 'RET' as the endogenous variable and 'PRC' as exogenous
endog = model_data[['PRC']]
exog = model_data= [['VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'TURNOVER', 'MARKET_CAP', 'TRAN_COST', 'ASK', 'BID', 'day_of_week',
            'day_of_month', 'month', 'is_month_start', 'is_month_end', 'year', 'week', 'RET_ema_12', 'RET_ema_26',
            'RET_skew_10', 'RET_kurtosis_10', 'RET_entropy_10', 'RSI', 'OBV', 'RET_lag_1', 'RET_lag_7', 'RET_lag_14',
            'RET_lag_30', 'RET_mean_7d', 'RET_std_7d', 'RET_mean_14d', 'RET_std_14d', 'RET_mean_30d', 'RET_std_30d']]



# Differencing
#endog_diff = endog.diff().dropna()
#exog_diff = exog.diff().dropna()

# Align the datasets
endog = endog.iloc[1:]
exog = exog.iloc[1:]

# Fit a simple VAR model
model = VAR(endog)
results = model.fit(1)
print(results.summary())


AttributeError: 'list' object has no attribute 'diff'